<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

#Data process

In [ ]:
From 1ce16998e092ecffcc5fa2cfffee6544c35fb3f2 Mon Sep 17 00:00:00 2001
From: zhengjun10 <zhengjun10@huawei.com>
Date: Sat, 5 Jun 2021 14:48:11 +0800
Subject: [PATCH] master code standard rectification

---
 .../export_models/models/densenet_train_export.py  | 39 +++++++++++++++++++---
 .../lite/examples/export_models/models/effnet.py   | 29 ++++++++++------
 .../export_models/models/effnet_train_export.py    |  6 ++--
 .../models/effnet_tune_train_export.py             | 12 ++++---
 .../export_models/models/googlenet_train_export.py |  6 ++--
 .../export_models/models/lenet_train_export.py     |  6 ++--
 .../examples/export_models/models/mini_alexnet.py  |  3 ++
 .../models/mini_alexnet_train_export.py            |  6 ++--
 .../models/mobilenetv1_train_export.py             |  6 ++--
 .../models/mobilenetv2_train_export.py             |  6 ++--
 .../models/mobilenetv3_train_export.py             |  6 ++--
 .../export_models/models/nin_train_export.py       |  6 ++--
 .../export_models/models/resnet_train_export.py    |  6 ++--
 .../models/shufflenetv2_train_export.py            |  6 ++--
 .../examples/export_models/models/train_utils.py   | 20 ++++++-----
 .../export_models/models/vgg_train_export.py       |  6 ++--
 .../export_models/models/xception_train_export.py  |  6 ++--
 .../examples/train_lenet/model/lenet_export.py     |  4 +--
 .../lite/examples/train_lenet/model/train_utils.py |  5 +--
 .../lite/examples/train_lenet/src/net_runner.cc    | 31 +++++++++++------
 .../examples/transfer_learning/model/effnet.py     | 16 +++++++--
 .../transfer_learning/model/train_utils.py         |  4 +--
 .../model/transfer_learning_export.py              |  4 +--
 .../lite/examples/transfer_learning/src/dataset.cc | 20 +++++++----
 .../examples/transfer_learning/src/net_runner.cc   | 18 ++++++----
 mindspore/lite/include/train/accuracy_metrics.h    |  1 +
 .../train/classification_train_accuracy_monitor.h  |  5 ++-
 mindspore/lite/src/huffman_decode.cc               | 11 +++---
 mindspore/lite/src/huffman_decode.h                |  2 +-
 .../lite/src/ops/populate/arithmetic_populate.cc   |  2 ++
 .../lite/src/ops/populate/bias_grad_populate.cc    | 39 ----------------------
 mindspore/lite/src/train/accuracy_metrics.cc       |  2 --
 mindspore/lite/src/train/accuracy_monitor.cc       |  2 --
 .../train/classification_train_accuracy_monitor.cc | 34 +++++++------------
 mindspore/lite/src/train/loss_monitor.cc           |  6 ++--
 mindspore/lite/src/train/lr_scheduler.cc           |  2 --
 mindspore/lite/src/train/train_loop.cc             | 27 ++-------------
 mindspore/lite/src/train/train_loop.h              |  1 -
 .../lite/src/train/train_populate_parameter_v0.cc  |  2 --
 mindspore/lite/src/weight_decoder.cc               |  2 +-
 40 files changed, 207 insertions(+), 208 deletions(-)
 delete mode 100644 mindspore/lite/src/ops/populate/bias_grad_populate.cc

diff --git a/mindspore/lite/examples/export_models/models/densenet_train_export.py b/mindspore/lite/examples/export_models/models/densenet_train_export.py
index dd46e32..a8c85a3 100644
--- a/mindspore/lite/examples/export_models/models/densenet_train_export.py
+++ b/mindspore/lite/examples/export_models/models/densenet_train_export.py
@@ -17,14 +17,43 @@
 import sys
 import os
 import numpy as np
-from train_utils import SaveInOut, TrainWrap
+from train_utils import save_inout, train_wrap
 import mindspore.common.dtype as mstype
 from mindspore import context, Tensor, nn
 from mindspore.train.serialization import export
+<<<<<<< Updated upstream
+<<<<<<< Updated upstream
+<<<<<<< Updated upstream
+<<<<<<< Updated upstream
+from src.network.densenet import DenseNet121
+sys.path.append(os.environ['CLOUD_MODEL_ZOO'] + 'official/cv/densenet121/')
+#pylint: disable=wrong-import-position
+
 
-sys.path.append(os.environ['CLOUD_MODEL_ZOO'] + 'official/cv/densenet/')
+=======
+=======
+>>>>>>> Stashed changes
+=======
+>>>>>>> Stashed changes
+from official.cv.densenet121.src.network.densenet import DenseNet121
+sys.path.append(os.environ['CLOUD_MODEL_ZOO'] + 'official/cv/densenet121/')
 #pylint: disable=wrong-import-position
-from src.network.densenet import DenseNet121
+
+<<<<<<< Updated upstream
+<<<<<<< Updated upstream
+>>>>>>> Stashed changes
+=======
+from official.cv.densenet121.src.network.densenet import DenseNet121
+sys.path.append(os.environ['CLOUD_MODEL_ZOO'] + 'official/cv/densenet121/')
+#pylint: disable=wrong-import-position
+
+>>>>>>> Stashed changes
+=======
+>>>>>>> Stashed changes
+=======
+>>>>>>> Stashed changes
+
+
 
 
 context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU", save_graphs=False)
@@ -33,7 +62,7 @@ n = DenseNet121(num_classes=10)
 loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=False)
 optimizer = nn.SGD(n.trainable_params(), learning_rate=0.001, momentum=0.9, dampening=0.0, weight_decay=0.0,
                    nesterov=True, loss_scale=0.9)
-net = TrainWrap(n, loss_fn, optimizer)
+net = train_wrap(n, loss_fn, optimizer)
 
 batch = 2
 x = Tensor(np.random.randn(batch, 3, 224, 224), mstype.float32)
@@ -41,4 +70,4 @@ label = Tensor(np.zeros([batch, 10]).astype(np.float32))
 export(net, x, label, file_name="mindir/densenet_train", file_format='MINDIR')
 
 if len(sys.argv) > 1:
-    SaveInOut(sys.argv[1] + "densenet", x, label, n, net)
+    save_inout(sys.argv[1] + "densenet", x, label, n, net)
diff --git a/mindspore/lite/examples/export_models/models/effnet.py b/mindspore/lite/examples/export_models/models/effnet.py
index 4971757..fc49872 100755
--- a/mindspore/lite/examples/export_models/models/effnet.py
+++ b/mindspore/lite/examples/export_models/models/effnet.py
@@ -20,6 +20,7 @@ from mindspore.ops import operations as P
 from mindspore.common.initializer import TruncatedNormal
 from mindspore import Tensor
 
+
 def weight_variable():
     """weight initial"""
     return TruncatedNormal(0.02)
@@ -40,6 +41,7 @@ def _make_value_divisible(value, factor, min_value=None):
         new_value += factor
     return new_value
 
+
 class Swish(nn.Cell):
     def __init__(self):
         super().__init__()
@@ -58,7 +60,8 @@ class AdaptiveAvgPool(nn.Cell):
         self.output_size = output_size
 
     def construct(self, x):
-        return self.mean(x, (2, 3)) ## This is not a general case
+        return self.mean(x, (2, 3)) # This is not a general case
+
 
 class SELayer(nn.Cell):
     """SELayer"""
@@ -74,24 +77,27 @@ class SELayer(nn.Cell):
         self.act2 = nn.Sigmoid()
 
     def construct(self, x):
-        o = self.avg_pool(x) #.view(b,c)
+        o = self.avg_pool(x) # .view(b,c)
         o = self.conv_reduce(o)
         o = self.act1(o)
         o = self.conv_expand(o)
-        o = self.act2(o) #.view(b, c, 1,1)
+        o = self.act2(o) # .view(b, c, 1,1)
         return x * o
 
+
 class DepthwiseSeparableConv(nn.Cell):
     """DepthwiseSeparableConv"""
     def __init__(self, in_chs, out_chs, dw_kernel_size=3, stride=1, noskip=False, se_ratio=0.0, drop_connect_rate=0.0):
         super().__init__()
-        assert stride in [1, 2]
+        if stride not in [1, 2]:
+            print("ERROR stride param")
+            return
         self.has_residual = (stride == 1 and in_chs == out_chs) and not noskip
         self.drop_connect_rate = drop_connect_rate
 
         self.conv_dw = nn.Conv2d(in_channels=in_chs, out_channels=in_chs, kernel_size=dw_kernel_size, stride=stride,
                                  pad_mode="pad", padding=1, has_bias=False, group=in_chs)
-        self.bn1 = nn.BatchNorm2d(in_chs, eps=0.001) #,momentum=0.1)
+        self.bn1 = nn.BatchNorm2d(in_chs, eps=0.001) # momentum=0.1)
         self.act1 = Swish()
 
        # Squeeze-and-excitation
@@ -101,7 +107,7 @@ class DepthwiseSeparableConv(nn.Cell):
             print("ERRRRRORRRR -- not prepared for this one\n")
 
         self.conv_pw = nn.Conv2d(in_channels=in_chs, out_channels=out_chs, kernel_size=1, stride=stride, has_bias=False)
-        self.bn2 = nn.BatchNorm2d(out_chs, eps=0.001) #,momentum=0.1)
+        self.bn2 = nn.BatchNorm2d(out_chs, eps=0.001) # momentum=0.1)
 
     def construct(self, x):
         """construct"""
@@ -120,12 +126,13 @@ class DepthwiseSeparableConv(nn.Cell):
             x += residual
         return x
 
+
 def conv_3x3_bn(inp, oup, stride):
     weight = weight_variable()
     return nn.SequentialCell([
         nn.Conv2d(in_channels=inp, out_channels=oup, kernel_size=3, stride=stride, padding=1, weight_init=weight,
                   has_bias=False, pad_mode='pad'),
-        nn.BatchNorm2d(oup, eps=0.001),  #, momentum=0.1),
+        nn.BatchNorm2d(oup, eps=0.001),  # momentum=0.1),
         nn.HSwish()])
 
 
@@ -142,7 +149,9 @@ class InvertedResidual(nn.Cell):
     """InvertedResidual"""
     def __init__(self, in_chs, out_chs, kernel_size, stride, padding, expansion, se_ratio):
         super().__init__()
-        assert stride in [1, 2]
+        if stride not in [1, 2]:
+            print("ERROR stride param")
+            return
         mid_chs: int = _make_value_divisible(in_chs * expansion, 1)
         self.has_residual = (in_chs == out_chs and stride == 1)
         self.drop_connect_rate = 0
@@ -210,7 +219,7 @@ class EfficientNet(nn.Cell):
 
         self.conv_stem = nn.Conv2d(in_channels=3, out_channels=stem_size, kernel_size=3, stride=2, has_bias=False)
 
-        self.bn1 = nn.BatchNorm2d(stem_size, eps=0.001) #momentum=0.1)
+        self.bn1 = nn.BatchNorm2d(stem_size, eps=0.001) # momentum=0.1)
         self.act1 = Swish()
         in_chs = stem_size
 
@@ -240,7 +249,7 @@ class EfficientNet(nn.Cell):
         self.blocks = nn.SequentialCell(layers)
 
         self.conv_head = nn.Conv2d(in_channels=320, out_channels=self.num_features_, kernel_size=1)
-        self.bn2 = nn.BatchNorm2d(self.num_features_, eps=0.001) #,momentum=0.1)
+        self.bn2 = nn.BatchNorm2d(self.num_features_, eps=0.001) # momentum=0.1)
         self.act2 = Swish()
         self.global_pool = AdaptiveAvgPool(output_size=(1, 1))
         self.classifier = nn.Dense(self.num_features_, num_classes)
diff --git a/mindspore/lite/examples/export_models/models/effnet_train_export.py b/mindspore/lite/examples/export_models/models/effnet_train_export.py
index bf341f2..3384cc2 100644
--- a/mindspore/lite/examples/export_models/models/effnet_train_export.py
+++ b/mindspore/lite/examples/export_models/models/effnet_train_export.py
@@ -16,7 +16,7 @@
 
 import sys
 import numpy as np
-from train_utils import SaveInOut, TrainWrap
+from train_utils import save_inout, train_wrap
 from effnet import effnet
 import mindspore.common.dtype as mstype
 from mindspore import context, Tensor, nn
@@ -28,11 +28,11 @@ n = effnet(num_classes=10)
 loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=False)
 optimizer = nn.SGD(n.trainable_params(), learning_rate=0.01, momentum=0.9, dampening=0.0, weight_decay=0.0,
                    nesterov=True, loss_scale=1.0)
-net = TrainWrap(n, loss_fn, optimizer)
+net = train_wrap(n, loss_fn, optimizer)
 
 x = Tensor(np.random.randn(2, 3, 224, 224), mstype.float32)
 label = Tensor(np.zeros([2, 10]).astype(np.float32))
 export(net, x, label, file_name="mindir/effnet_train", file_format='MINDIR')
 
 if len(sys.argv) > 1:
-    SaveInOut(sys.argv[1] + "effnet", x, label, n, net)
+    save_inout(sys.argv[1] + "effnet", x, label, n, net)
diff --git a/mindspore/lite/examples/export_models/models/effnet_tune_train_export.py b/mindspore/lite/examples/export_models/models/effnet_tune_train_export.py
index 2b21ee8..3e61b44 100644
--- a/mindspore/lite/examples/export_models/models/effnet_tune_train_export.py
+++ b/mindspore/lite/examples/export_models/models/effnet_tune_train_export.py
@@ -17,7 +17,7 @@
 import sys
 from os import path
 import numpy as np
-from train_utils import TrainWrap, SaveT
+from train_utils import train_wrap, save_t
 from effnet import effnet
 import mindspore.common.dtype as mstype
 from mindspore import context, Tensor, nn
@@ -26,11 +26,13 @@ from mindspore.common.parameter import ParameterTuple
 
 context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU", save_graphs=False)
 
+
 class TransferNet(nn.Cell):
     def __init__(self, backbone, head):
         super().__init__(TransferNet)
         self.backbone = backbone
         self.head = head
+
     def construct(self, x):
         x = self.backbone(x)
         x = self.head(x)
@@ -56,7 +58,7 @@ trainable_weights_list.extend(n.head.trainable_params())
 trainable_weights = ParameterTuple(trainable_weights_list)
 sgd = nn.SGD(trainable_weights, learning_rate=0.01, momentum=0.9,
              dampening=0.01, weight_decay=0.0, nesterov=False, loss_scale=1.0)
-net = TrainWrap(n, optimizer=sgd, weights=trainable_weights)
+net = train_wrap(n, optimizer=sgd, weights=trainable_weights)
 
 BATCH_SIZE = 8
 X = Tensor(np.random.randn(BATCH_SIZE, 3, 224, 224), mstype.float32)
@@ -66,10 +68,10 @@ export(net, X, label, file_name="mindir/effnet_tune_train", file_format='MINDIR'
 if len(sys.argv) > 1:
     name_prefix = sys.argv[1] + "effnet_tune"
     x_name = name_prefix + "_input1.bin"
-    SaveT(Tensor(X.asnumpy().transpose(0, 2, 3, 1)), x_name)
+    save_t(Tensor(X.asnumpy().transpose(0, 2, 3, 1)), x_name)
 
     l_name = name_prefix + "_input2.bin"
-    SaveT(label, l_name)
+    save_t(label, l_name)
 
     #train network
     n.head.set_train(True)
@@ -80,4 +82,4 @@ if len(sys.argv) > 1:
     n.set_train(False)
     y = n(X)
     y_name = name_prefix + "_output1.bin"
-    SaveT(y, y_name)
+    save_t(y, y_name)
diff --git a/mindspore/lite/examples/export_models/models/googlenet_train_export.py b/mindspore/lite/examples/export_models/models/googlenet_train_export.py
index c2ddcc2..91a0062 100644
--- a/mindspore/lite/examples/export_models/models/googlenet_train_export.py
+++ b/mindspore/lite/examples/export_models/models/googlenet_train_export.py
@@ -16,7 +16,7 @@
 
 import sys
 import numpy as np
-from train_utils import SaveInOut, TrainWrap
+from train_utils import save_inout, train_wrap
 from official.cv.googlenet.src.googlenet import GoogleNet
 import mindspore.common.dtype as mstype
 from mindspore import context, Tensor, nn
@@ -28,7 +28,7 @@ n = GoogleNet(num_classes=10)
 loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=False)
 optimizer = nn.SGD(n.trainable_params(), learning_rate=0.01, momentum=0.9, dampening=0.0, weight_decay=5e-4,
                    nesterov=True, loss_scale=0.9)
-net = TrainWrap(n, loss_fn, optimizer)
+net = train_wrap(n, loss_fn, optimizer)
 
 batch = 2
 x = Tensor(np.random.randn(batch, 3, 224, 224), mstype.float32)
@@ -36,4 +36,4 @@ label = Tensor(np.zeros([batch, 10]).astype(np.float32))
 export(net, x, label, file_name="mindir/googlenet_train", file_format='MINDIR')
 
 if len(sys.argv) > 1:
-    SaveInOut(sys.argv[1] + "googlenet", x, label, n, net)
+    save_inout(sys.argv[1] + "googlenet", x, label, n, net)
diff --git a/mindspore/lite/examples/export_models/models/lenet_train_export.py b/mindspore/lite/examples/export_models/models/lenet_train_export.py
index 1b7dfda..4e03aab 100644
--- a/mindspore/lite/examples/export_models/models/lenet_train_export.py
+++ b/mindspore/lite/examples/export_models/models/lenet_train_export.py
@@ -16,7 +16,7 @@
 
 import sys
 import numpy as np
-from train_utils import SaveInOut, TrainWrap
+from train_utils import save_inout, train_wrap
 from official.cv.lenet.src.lenet import LeNet5
 import mindspore.common.dtype as mstype
 from mindspore import context, Tensor, nn
@@ -28,11 +28,11 @@ n = LeNet5()
 loss_fn = nn.MSELoss()
 optimizer = nn.Adam(n.trainable_params(), learning_rate=1e-2, beta1=0.5, beta2=0.7, eps=1e-2, use_locking=True,
                     use_nesterov=False, weight_decay=0.0, loss_scale=0.3)
-net = TrainWrap(n, loss_fn, optimizer)
+net = train_wrap(n, loss_fn, optimizer)
 
 x = Tensor(np.random.randn(32, 1, 32, 32), mstype.float32)
 label = Tensor(np.zeros([32, 10]).astype(np.float32))
 export(net, x, label, file_name="mindir/lenet_train", file_format='MINDIR')
 
 if len(sys.argv) > 1:
-    SaveInOut(sys.argv[1] + "lenet", x, label, n, net, sparse=False)
+    save_inout(sys.argv[1] + "lenet", x, label, n, net, sparse=False)
diff --git a/mindspore/lite/examples/export_models/models/mini_alexnet.py b/mindspore/lite/examples/export_models/models/mini_alexnet.py
index 9a8b828..e6008fa 100644
--- a/mindspore/lite/examples/export_models/models/mini_alexnet.py
+++ b/mindspore/lite/examples/export_models/models/mini_alexnet.py
@@ -17,13 +17,16 @@
 import mindspore.nn as nn
 from mindspore.ops import operations as P
 
+
 def conv(in_channels, out_channels, kernel_size, stride=1, padding=0, pad_mode="valid", has_bias=True):
     return nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                      has_bias=has_bias, pad_mode=pad_mode)
 
+
 def fc_with_initialize(input_channels, out_channels, has_bias=True):
     return nn.Dense(input_channels, out_channels, has_bias=has_bias)
 
+
 class AlexNet(nn.Cell):
     """
     Alexnet
diff --git a/mindspore/lite/examples/export_models/models/mini_alexnet_train_export.py b/mindspore/lite/examples/export_models/models/mini_alexnet_train_export.py
index 544daad..1b9a82d 100644
--- a/mindspore/lite/examples/export_models/models/mini_alexnet_train_export.py
+++ b/mindspore/lite/examples/export_models/models/mini_alexnet_train_export.py
@@ -16,7 +16,7 @@
 
 import sys
 import numpy as np
-from train_utils import SaveInOut, TrainWrap
+from train_utils import save_inout, train_wrap
 from mini_alexnet import AlexNet
 from mindspore import context, Tensor, nn
 from mindspore.train.serialization import export
@@ -31,11 +31,11 @@ n = AlexNet(phase='test')
 loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=False)
 optimizer = nn.Adam(n.trainable_params(), learning_rate=1e-3, beta1=0.9, beta2=0.999, eps=1e-8, use_locking=False,
                     use_nesterov=False, weight_decay=0.0, loss_scale=1.0)
-net = TrainWrap(n, loss_fn, optimizer)
+net = train_wrap(n, loss_fn, optimizer)
 
 x = Tensor(np.ones([batch, 1, 32, 32]).astype(np.float32) * 0.01)
 label = Tensor(np.zeros([batch, number_of_classes]).astype(np.float32))
 export(net, x, label, file_name="mindir/mini_alexnet_train", file_format='MINDIR')
 
 if len(sys.argv) > 1:
-    SaveInOut(sys.argv[1] + "mini_alexnet", x, label, n, net, sparse=False)
+    save_inout(sys.argv[1] + "mini_alexnet", x, label, n, net, sparse=False)
diff --git a/mindspore/lite/examples/export_models/models/mobilenetv1_train_export.py b/mindspore/lite/examples/export_models/models/mobilenetv1_train_export.py
index f668a96..b3b26de 100644
--- a/mindspore/lite/examples/export_models/models/mobilenetv1_train_export.py
+++ b/mindspore/lite/examples/export_models/models/mobilenetv1_train_export.py
@@ -16,7 +16,7 @@
 
 import sys
 import numpy as np
-from train_utils import SaveInOut, TrainWrap
+from train_utils import save_inout, train_wrap
 from official.cv.mobilenetv1.src.mobilenet_v1 import MobileNetV1
 import mindspore.common.dtype as mstype
 from mindspore import context, Tensor, nn
@@ -28,7 +28,7 @@ n = MobileNetV1(10)
 loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=False)
 optimizer = nn.SGD(n.trainable_params(), learning_rate=1e-2, momentum=0.9, dampening=0.1, weight_decay=0.0,
                    nesterov=False, loss_scale=1.0)
-net = TrainWrap(n, loss_fn, optimizer)
+net = train_wrap(n, loss_fn, optimizer)
 
 batch = 2
 x = Tensor(np.random.randn(batch, 3, 224, 224), mstype.float32)
@@ -37,4 +37,4 @@ label = Tensor(np.zeros([batch, 10]).astype(np.float32))
 export(net, x, label, file_name="mindir/mobilenetv1_train", file_format='MINDIR')
 
 if len(sys.argv) > 1:
-    SaveInOut(sys.argv[1] + "mobilenetv1", x, label, n, net)
+    save_inout(sys.argv[1] + "mobilenetv1", x, label, n, net)
diff --git a/mindspore/lite/examples/export_models/models/mobilenetv2_train_export.py b/mindspore/lite/examples/export_models/models/mobilenetv2_train_export.py
index 8f1d543..0433063 100644
--- a/mindspore/lite/examples/export_models/models/mobilenetv2_train_export.py
+++ b/mindspore/lite/examples/export_models/models/mobilenetv2_train_export.py
@@ -16,7 +16,7 @@
 
 import sys
 import numpy as np
-from train_utils import SaveInOut, TrainWrap
+from train_utils import save_inout, train_wrap
 from official.cv.mobilenetv2.src.mobilenetV2 import MobileNetV2Backbone, MobileNetV2Head, mobilenet_v2
 import mindspore.common.dtype as mstype
 from mindspore import context, Tensor, nn
@@ -31,11 +31,11 @@ n = mobilenet_v2(backbone_net, head_net)
 
 loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=False)
 optimizer = nn.Momentum(n.trainable_params(), 0.01, 0.9, use_nesterov=False)
-net = TrainWrap(n, loss_fn, optimizer)
+net = train_wrap(n, loss_fn, optimizer)
 
 x = Tensor(np.random.randn(batch, 3, 224, 224), mstype.float32)
 label = Tensor(np.zeros([batch, 10]).astype(np.float32))
 export(net, x, label, file_name="mindir/mobilenetv2_train", file_format='MINDIR')
 
 if len(sys.argv) > 1:
-    SaveInOut(sys.argv[1] + "mobilenetv2", x, label, n, net, sparse=False)
+    save_inout(sys.argv[1] + "mobilenetv2", x, label, n, net, sparse=False)
diff --git a/mindspore/lite/examples/export_models/models/mobilenetv3_train_export.py b/mindspore/lite/examples/export_models/models/mobilenetv3_train_export.py
index 26a6671..1f75252 100644
--- a/mindspore/lite/examples/export_models/models/mobilenetv3_train_export.py
+++ b/mindspore/lite/examples/export_models/models/mobilenetv3_train_export.py
@@ -16,7 +16,7 @@
 
 import sys
 import numpy as np
-from train_utils import SaveInOut, TrainWrap
+from train_utils import save_inout, train_wrap
 from official.cv.mobilenetv3.src.mobilenetV3 import mobilenet_v3_small
 import mindspore.common.dtype as mstype
 from mindspore import context, Tensor, nn
@@ -28,7 +28,7 @@ n = mobilenet_v3_small(num_classes=10)
 loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=False, reduction='mean')
 optimizer = nn.Adam(n.trainable_params(), learning_rate=1e-3, beta1=0.5, beta2=0.7, eps=1e-2, use_locking=True,
                     use_nesterov=False, weight_decay=0.1, loss_scale=0.3)
-net = TrainWrap(n, loss_fn, optimizer)
+net = train_wrap(n, loss_fn, optimizer)
 
 batch = 2
 x = Tensor(np.random.randn(batch, 3, 224, 224), mstype.float32)
@@ -36,4 +36,4 @@ label = Tensor(np.zeros([batch, 10]).astype(np.float32))
 export(net, x, label, file_name="mindir/mobilenetv3_train", file_format='MINDIR')
 
 if len(sys.argv) > 1:
-    SaveInOut(sys.argv[1] + "mobilenetv3", x, label, n, net, sparse=False)
+    save_inout(sys.argv[1] + "mobilenetv3", x, label, n, net, sparse=False)
diff --git a/mindspore/lite/examples/export_models/models/nin_train_export.py b/mindspore/lite/examples/export_models/models/nin_train_export.py
index 72ccc5e..786f739 100644
--- a/mindspore/lite/examples/export_models/models/nin_train_export.py
+++ b/mindspore/lite/examples/export_models/models/nin_train_export.py
@@ -16,7 +16,7 @@
 
 import sys
 import numpy as np
-from train_utils import SaveInOut, TrainWrap
+from train_utils import save_inout, train_wrap
 from NetworkInNetwork import NiN
 import mindspore.common.dtype as mstype
 from mindspore import context, Tensor, nn
@@ -28,7 +28,7 @@ n = NiN(num_classes=10)
 loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
 optimizer = nn.SGD(n.trainable_params(), learning_rate=0.01, momentum=0.9, dampening=0.0, weight_decay=5e-4,
                    nesterov=True, loss_scale=0.9)
-net = TrainWrap(n, loss_fn, optimizer)
+net = train_wrap(n, loss_fn, optimizer)
 
 batch = 2
 x = Tensor(np.random.randn(batch, 3, 32, 32), mstype.float32)
@@ -36,4 +36,4 @@ label = Tensor(np.zeros([batch]).astype(np.int32))
 export(net, x, label, file_name="mindir/nin_train", file_format='MINDIR')
 
 if len(sys.argv) > 1:
-    SaveInOut(sys.argv[1] + "nin", x, label, n, net)
+    save_inout(sys.argv[1] + "nin", x, label, n, net)
diff --git a/mindspore/lite/examples/export_models/models/resnet_train_export.py b/mindspore/lite/examples/export_models/models/resnet_train_export.py
index c0dbe90..c18bcf3 100644
--- a/mindspore/lite/examples/export_models/models/resnet_train_export.py
+++ b/mindspore/lite/examples/export_models/models/resnet_train_export.py
@@ -16,7 +16,7 @@
 
 import sys
 import numpy as np
-from train_utils import SaveInOut, TrainWrap
+from train_utils import save_inout, train_wrap
 from official.cv.resnet.src.resnet import resnet50
 import mindspore.common.dtype as mstype
 from mindspore import context, Tensor, nn
@@ -29,11 +29,11 @@ n = resnet50(class_num=10)
 loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=False)
 optimizer = nn.SGD(n.trainable_params(), learning_rate=0.01, momentum=0.9, dampening=0.0, weight_decay=0.0,
                    nesterov=True, loss_scale=1.0)
-net = TrainWrap(n, loss_fn, optimizer)
+net = train_wrap(n, loss_fn, optimizer)
 
 x = Tensor(np.random.randn(batch, 3, 224, 224), mstype.float32)
 label = Tensor(np.zeros([batch, 10]).astype(np.float32))
 export(net, x, label, file_name="mindir/resnet_train", file_format='MINDIR')
 
 if len(sys.argv) > 1:
-    SaveInOut(sys.argv[1] + "resnet", x, label, n, net)
+    save_inout(sys.argv[1] + "resnet", x, label, n, net)
diff --git a/mindspore/lite/examples/export_models/models/shufflenetv2_train_export.py b/mindspore/lite/examples/export_models/models/shufflenetv2_train_export.py
index 97aa4ec..bf76d48 100644
--- a/mindspore/lite/examples/export_models/models/shufflenetv2_train_export.py
+++ b/mindspore/lite/examples/export_models/models/shufflenetv2_train_export.py
@@ -16,7 +16,7 @@
 
 import sys
 import numpy as np
-from train_utils import SaveInOut, TrainWrap
+from train_utils import save_inout, train_wrap
 from official.cv.shufflenetv2.src.shufflenetv2 import ShuffleNetV2
 import mindspore.common.dtype as mstype
 from mindspore import context, Tensor, nn
@@ -28,7 +28,7 @@ n = ShuffleNetV2(n_class=10)
 loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=False)
 optimizer = nn.Momentum(n.trainable_params(), 0.01, 0.9, use_nesterov=False)
 
-net = TrainWrap(n, loss_fn, optimizer)
+net = train_wrap(n, loss_fn, optimizer)
 
 batch = 2
 x = Tensor(np.random.randn(batch, 3, 224, 224), mstype.float32)
@@ -36,4 +36,4 @@ label = Tensor(np.zeros([batch, 10]).astype(np.float32))
 export(net, x, label, file_name="mindir/shufflenetv2_train", file_format='MINDIR')
 
 if len(sys.argv) > 1:
-    SaveInOut(sys.argv[1] + "shufflenetv2", x, label, n, net)
+    save_inout(sys.argv[1] + "shufflenetv2", x, label, n, net)
diff --git a/mindspore/lite/examples/export_models/models/train_utils.py b/mindspore/lite/examples/export_models/models/train_utils.py
index e32fda1..5017b8f 100644
--- a/mindspore/lite/examples/export_models/models/train_utils.py
+++ b/mindspore/lite/examples/export_models/models/train_utils.py
@@ -14,11 +14,13 @@
 # ============================================================================
 """train_utils."""
 
+import os
 from mindspore import nn, Tensor
 from mindspore.common.parameter import ParameterTuple
 
-def TrainWrap(net, loss_fn=None, optimizer=None, weights=None):
-    """TrainWrap"""
+
+def train_wrap(net, loss_fn=None, optimizer=None, weights=None):
+    """train_wrap"""
     if loss_fn is None:
         loss_fn = nn.SoftmaxCrossEntropyWithLogits()
     loss_net = nn.WithLossCell(net, loss_fn)
@@ -32,22 +34,22 @@ def TrainWrap(net, loss_fn=None, optimizer=None, weights=None):
     return train_net
 
 
-def SaveT(t, file):
+def save_t(t, file):
     x = t.asnumpy()
     x.tofile(file)
 
 
-def SaveInOut(name, x, l, net, net_train, sparse=False, epoch=1):
-    """SaveInOut"""
+def save_inout(name, x, l, net, net_train, sparse=False, epoch=1):
+    """save_inout"""
     x_name = name + "_input1.bin"
     if sparse:
         x_name = name + "_input2.bin"
-    SaveT(Tensor(x.asnumpy().transpose(0, 2, 3, 1)), x_name)
+    save_t(Tensor(x.asnumpy().transpose(0, 2, 3, 1)), x_name)
 
     l_name = name + "_input2.bin"
     if sparse:
         l_name = name + "_input1.bin"
-    SaveT(l, l_name)
+    save_t(l, l_name)
 
     net.set_train(False)
     y = net(x)
@@ -62,10 +64,10 @@ def SaveInOut(name, x, l, net, net_train, sparse=False, epoch=1):
     if isinstance(y, tuple):
         i = 1
         for t in y:
-            with open(name + "_output" + str(i) + ".bin", 'w') as f:
+            with os.fdopen(name + "_output" + str(i) + ".bin", 'w') as f:
                 for j in t.asnumpy().flatten():
                     f.write(str(j)+' ')
             i = i + 1
     else:
         y_name = name + "_output1.bin"
-        SaveT(y, y_name)
+        save_t(y, y_name)
diff --git a/mindspore/lite/examples/export_models/models/vgg_train_export.py b/mindspore/lite/examples/export_models/models/vgg_train_export.py
index 0078252..c18b33c 100644
--- a/mindspore/lite/examples/export_models/models/vgg_train_export.py
+++ b/mindspore/lite/examples/export_models/models/vgg_train_export.py
@@ -16,7 +16,7 @@
 
 import sys
 import numpy as np
-from train_utils import SaveInOut, TrainWrap
+from train_utils import save_inout, train_wrap
 from official.cv.vgg16.src.vgg import vgg16
 import mindspore.common.dtype as mstype
 from mindspore import context, Tensor, nn
@@ -29,11 +29,11 @@ batch = 2
 n = vgg16(num_classes=10)
 loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=False)
 optimizer = nn.Momentum(n.trainable_params(), 0.01, 0.9, use_nesterov=False)
-net = TrainWrap(n, loss_fn, optimizer)
+net = train_wrap(n, loss_fn, optimizer)
 
 x = Tensor(np.random.randn(batch, 3, 224, 224), mstype.float32)
 label = Tensor(np.zeros([batch, 10]).astype(np.float32))
 export(net, x, label, file_name="mindir/vgg_train", file_format='MINDIR')
 
 if len(sys.argv) > 1:
-    SaveInOut(sys.argv[1] + "vgg", x, label, n, net)
+    save_inout(sys.argv[1] + "vgg", x, label, n, net)
diff --git a/mindspore/lite/examples/export_models/models/xception_train_export.py b/mindspore/lite/examples/export_models/models/xception_train_export.py
index 6b82b3b..e544d7e 100644
--- a/mindspore/lite/examples/export_models/models/xception_train_export.py
+++ b/mindspore/lite/examples/export_models/models/xception_train_export.py
@@ -16,7 +16,7 @@
 
 import sys
 import numpy as np
-from train_utils import SaveInOut, TrainWrap
+from train_utils import save_inout, train_wrap
 from official.cv.xception.src.Xception import Xception
 import mindspore.common.dtype as mstype
 from mindspore import context, Tensor, nn
@@ -31,7 +31,7 @@ n.dropout = nn.Dropout(keep_prob=1.0)
 loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=False)
 optimizer = nn.SGD(n.trainable_params(), learning_rate=0.01, momentum=0.9, dampening=0.0, weight_decay=0.0,
                    nesterov=True, loss_scale=1.0)
-net = TrainWrap(n, loss_fn, optimizer)
+net = train_wrap(n, loss_fn, optimizer)
 
 batch = 2
 x = Tensor(np.random.randn(batch, 3, 299, 299), mstype.float32)
@@ -39,4 +39,4 @@ label = Tensor(np.zeros([batch, 1000]).astype(np.float32))
 export(net, x, label, file_name="mindir/xception_train", file_format='MINDIR')
 
 if len(sys.argv) > 1:
-    SaveInOut(sys.argv[1] + "xception", x, label, n, net)
+    save_inout(sys.argv[1] + "xception", x, label, n, net)
diff --git a/mindspore/lite/examples/train_lenet/model/lenet_export.py b/mindspore/lite/examples/train_lenet/model/lenet_export.py
index 8a9cd7c..c774887 100644
--- a/mindspore/lite/examples/train_lenet/model/lenet_export.py
+++ b/mindspore/lite/examples/train_lenet/model/lenet_export.py
@@ -19,7 +19,7 @@ from mindspore import context, Tensor
 import mindspore.common.dtype as mstype
 from mindspore.train.serialization import export
 from lenet import LeNet5
-from train_utils import TrainWrap
+from train_utils import train_wrap
 
 n = LeNet5()
 n.set_train()
@@ -28,7 +28,7 @@ context.set_context(mode=context.PYNATIVE_MODE, device_target="CPU", save_graphs
 BATCH_SIZE = 32
 x = Tensor(np.ones((BATCH_SIZE, 1, 32, 32)), mstype.float32)
 label = Tensor(np.zeros([BATCH_SIZE]).astype(np.int32))
-net = TrainWrap(n)
+net = train_wrap(n)
 export(net, x, label, file_name="lenet_tod", file_format='MINDIR')
 
 print("finished exporting")
diff --git a/mindspore/lite/examples/train_lenet/model/train_utils.py b/mindspore/lite/examples/train_lenet/model/train_utils.py
index 9e8e3fa..9e3ad76 100644
--- a/mindspore/lite/examples/train_lenet/model/train_utils.py
+++ b/mindspore/lite/examples/train_lenet/model/train_utils.py
@@ -17,9 +17,10 @@
 import mindspore.nn as nn
 from mindspore.common.parameter import ParameterTuple
 
-def TrainWrap(net, loss_fn=None, optimizer=None, weights=None):
+
+def train_wrap(net, loss_fn=None, optimizer=None, weights=None):
     """
-    TrainWrap
+    train_wrap
     """
     if loss_fn is None:
         loss_fn = nn.SoftmaxCrossEntropyWithLogits(reduction='mean', sparse=True)
diff --git a/mindspore/lite/examples/train_lenet/src/net_runner.cc b/mindspore/lite/examples/train_lenet/src/net_runner.cc
index 1de4f72..4045bdb 100644
--- a/mindspore/lite/examples/train_lenet/src/net_runner.cc
+++ b/mindspore/lite/examples/train_lenet/src/net_runner.cc
@@ -44,10 +44,20 @@ using mindspore::lite::Model;
 using mindspore::session::TrainLoopCallBack;
 using mindspore::session::TrainLoopCallBackData;
 
+constexpr int kPrintNum = 10;
+constexpr float kScalePoint = 255.0f;
+constexpr int kBatchSize = 2;
+constexpr int kNCHWDims = 4;
+constexpr int kNCHWCDim = 2;
+constexpr int kPrintTimes = 100;
+constexpr int kSaveSteps = 1000;
+constexpr float kLearningRate = 0.7f;
 class Rescaler : public mindspore::session::TrainLoopCallBack {
  public:
   explicit Rescaler(float scale) : scale_(scale) {
-    if (scale_ == 0) scale_ = 1.0;
+    if (scale_ == 0) {
+      scale_ = 1.0;
+    }
   }
   ~Rescaler() override = default;
   void StepBegin(const mindspore::session::TrainLoopCallBackData &cb_data) override {
@@ -68,7 +78,7 @@ bool after_callback(const std::vector<mindspore::tensor::MSTensor *> &after_inpu
   for (size_t i = 0; i < after_inputs.size(); i++) {
     int num2p = (after_inputs.at(i)->ElementsNum());
     printf("in%zu(%d): ", i, num2p);
-    if (num2p > 10) num2p = 10;
+    if (num2p > kPrintNum) num2p = kPrintNum;
     if (after_inputs.at(i)->data_type() == mindspore::kNumberTypeInt32) {
       auto d = reinterpret_cast<int *>(after_inputs.at(i)->MutableData());
       for (int j = 0; j < num2p; j++) printf("%d, ", d[j]);
@@ -101,8 +111,7 @@ void NetRunner::InitAndFigureInputs() {
   context.thread_num_ = 2;
 
   session_ = mindspore::session::LiteSession::CreateTrainSession(ms_file_, &context, true);
-
-  MS_ASSERT(nullptr != session_);
+  MS_ASSERT(session_ != nullptr);
   loop_ = mindspore::session::TrainLoop::CreateTrainLoop(session_);
 
   if (verbose_) {
@@ -115,10 +124,10 @@ void NetRunner::InitAndFigureInputs() {
   auto inputs = session_->GetInputs();
   MS_ASSERT(inputs.size() > 1);
   auto nhwc_input_dims = inputs.at(0)->shape();
-  MS_ASSERT(nhwc_input_dims.size() == 4);
+  MS_ASSERT(nhwc_input_dims.size() == kNCHWDims);
   batch_size_ = nhwc_input_dims.at(0);
   h_ = nhwc_input_dims.at(1);
-  w_ = nhwc_input_dims.at(2);
+  w_ = nhwc_input_dims.at(kNCHWCDim);
 }
 
 float NetRunner::CalculateAccuracy(int max_tests) {
@@ -131,7 +140,7 @@ float NetRunner::CalculateAccuracy(int max_tests) {
   test_ds_ = test_ds_->Map({&typecast}, {"label"});
   test_ds_ = test_ds_->Batch(batch_size_, true);
 
-  Rescaler rescale(255.0);
+  Rescaler rescale(kScalePoint);
 
   loop_->Eval(test_ds_.get(), std::vector<TrainLoopCallBack *>{&rescale});
   std::cout << "Eval Accuracy is " << acc_metrics_->Eval() << std::endl;
@@ -162,13 +171,13 @@ int NetRunner::InitDB() {
 }
 
 int NetRunner::TrainLoop() {
-  struct mindspore::lite::StepLRLambda step_lr_lambda(1, 0.7);
+  struct mindspore::lite::StepLRLambda step_lr_lambda(1, kLearningRate);
   mindspore::lite::LRScheduler step_lr_sched(mindspore::lite::StepLRLambda, static_cast<void *>(&step_lr_lambda), 1);
 
-  mindspore::lite::LossMonitor lm(100);
+  mindspore::lite::LossMonitor lm(kPrintTimes);
   mindspore::lite::ClassificationTrainAccuracyMonitor am(1);
-  mindspore::lite::CkptSaver cs(1000, std::string("lenet"));
-  Rescaler rescale(255.0);
+  mindspore::lite::CkptSaver cs(kSaveSteps, std::string("lenet"));
+  Rescaler rescale(kScalePoint);
 
   loop_->Train(epochs_, train_ds_.get(), std::vector<TrainLoopCallBack *>{&rescale, &lm, &cs, &am, &step_lr_sched});
   return 0;
diff --git a/mindspore/lite/examples/transfer_learning/model/effnet.py b/mindspore/lite/examples/transfer_learning/model/effnet.py
index 8ed066f..eba29b5 100755
--- a/mindspore/lite/examples/transfer_learning/model/effnet.py
+++ b/mindspore/lite/examples/transfer_learning/model/effnet.py
@@ -44,6 +44,7 @@ class Swish(nn.Cell):
         m = x*s
         return m
 
+
 class AdaptiveAvgPool(nn.Cell):
     def __init__(self, output_size=None):
         super().__init__(AdaptiveAvgPool)
@@ -53,6 +54,7 @@ class AdaptiveAvgPool(nn.Cell):
     def construct(self, x):
         return self.mean(x, (2, 3))
 
+
 class SELayer(nn.Cell):
     """
     SELayer
@@ -77,6 +79,7 @@ class SELayer(nn.Cell):
         o = self.act2(o)
         return x * o
 
+
 class DepthwiseSeparableConv(nn.Cell):
     """
     DepthwiseSeparableConv
@@ -84,7 +87,9 @@ class DepthwiseSeparableConv(nn.Cell):
     def __init__(self, in_chs, out_chs, dw_kernel_size=3,
                  stride=1, noskip=False, se_ratio=0.0, drop_connect_rate=0.0):
         super().__init__(DepthwiseSeparableConv)
-        assert stride in [1, 2]
+        if stride not in [1, 2]:
+            print("ERROR")
+            return
         self.has_residual = (stride == 1 and in_chs == out_chs) and not noskip
         self.drop_connect_rate = drop_connect_rate
 
@@ -117,6 +122,7 @@ class DepthwiseSeparableConv(nn.Cell):
             x += residual
         return x
 
+
 def conv_3x3_bn(inp, oup, stride):
     weight = weight_variable()
     return nn.SequentialCell([
@@ -125,6 +131,7 @@ def conv_3x3_bn(inp, oup, stride):
         nn.BatchNorm2d(oup, eps=0.001),  # , momentum=0.1),
         nn.HSwish()])
 
+
 def conv_1x1_bn(inp, oup):
     weight = weight_variable()
     return nn.SequentialCell([
@@ -133,13 +140,16 @@ def conv_1x1_bn(inp, oup):
         nn.BatchNorm2d(oup, eps=0.001),
         nn.HSwish()])
 
+
 class InvertedResidual(nn.Cell):
     """
     InvertedResidual
     """
     def __init__(self, in_chs, out_chs, kernel_size, stride, padding, expansion, se_ratio):
         super().__init__(InvertedResidual)
-        assert stride in [1, 2]
+        if stride not in [1, 2]:
+            print("ERROR")
+            return
         mid_chs: int = _make_divisible(in_chs * expansion, 1)
         self.has_residual = (in_chs == out_chs and stride == 1)
         self.drop_connect_rate = 0
@@ -194,6 +204,7 @@ class InvertedResidual(nn.Cell):
             x += residual
         return x
 
+
 class EfficientNet(nn.Cell):
     """
     EfficientNet
@@ -295,6 +306,7 @@ class EfficientNet(nn.Cell):
             elif isinstance(m, nn.Dense):
                 init_linear_weight(m)
 
+
 def effnet(**kwargs):
     """
     Constructs a EfficientNet model
diff --git a/mindspore/lite/examples/transfer_learning/model/train_utils.py b/mindspore/lite/examples/transfer_learning/model/train_utils.py
index 550dd34..53585c8 100644
--- a/mindspore/lite/examples/transfer_learning/model/train_utils.py
+++ b/mindspore/lite/examples/transfer_learning/model/train_utils.py
@@ -17,9 +17,9 @@
 import mindspore.nn as nn
 from mindspore.common.parameter import ParameterTuple
 
-def TrainWrap(net, loss_fn=None, optimizer=None, weights=None):
+def train_wrap(net, loss_fn=None, optimizer=None, weights=None):
     """
-    TrainWrap
+    train_wrap
     """
     if loss_fn is None:
         loss_fn = nn.SoftmaxCrossEntropyWithLogits(reduction='mean')
diff --git a/mindspore/lite/examples/transfer_learning/model/transfer_learning_export.py b/mindspore/lite/examples/transfer_learning/model/transfer_learning_export.py
index a6009c7..9d10cc9 100755
--- a/mindspore/lite/examples/transfer_learning/model/transfer_learning_export.py
+++ b/mindspore/lite/examples/transfer_learning/model/transfer_learning_export.py
@@ -19,7 +19,7 @@ import mindspore as M
 from mindspore.nn import Cell
 from mindspore.train.serialization import load_checkpoint, export
 from effnet import effnet
-from train_utils import TrainWrap
+from train_utils import train_wrap
 
 
 class TransferNet(Cell):
@@ -51,7 +51,7 @@ HEAD.bias.set_data(M.Tensor(np.zeros(HEAD.bias.data.shape, dtype="float32")))
 
 sgd = M.nn.SGD(HEAD.trainable_params(), learning_rate=0.015, momentum=0.9,
                dampening=0.01, weight_decay=0.0, nesterov=False, loss_scale=1.0)
-net = TrainWrap(HEAD, optimizer=sgd)
+net = train_wrap(HEAD, optimizer=sgd)
 backbone_out = M.Tensor(np.zeros([BATCH_SIZE, 1000]).astype(np.float32))
 export(net, backbone_out, label, file_name="transfer_learning_tod_head", file_format='MINDIR')
 
diff --git a/mindspore/lite/examples/transfer_learning/src/dataset.cc b/mindspore/lite/examples/transfer_learning/src/dataset.cc
index 7a0669b..2f8d3c2 100644
--- a/mindspore/lite/examples/transfer_learning/src/dataset.cc
+++ b/mindspore/lite/examples/transfer_learning/src/dataset.cc
@@ -50,6 +50,10 @@ float CH_MEAN[3] = {0.485, 0.456, 0.406};
 float CH_STD[3] = {0.229, 0.224, 0.225};
 
 using LabelId = std::map<std::string, int>;
+constexpr int kClassNum = 10;
+constexpr int kBGRDim = 2;
+constexpr float kRGBMAX = 255.0f;
+constexpr int kRGBDims = 3;
 
 static char *ReadBitmapFile(const std::string &filename, size_t *size) {
   MS_ASSERT(size != nullptr);
@@ -78,7 +82,7 @@ static char *ReadBitmapFile(const std::string &filename, size_t *size) {
 
   ifs.read(reinterpret_cast<char *>(bmp_image), bitmap_header.image_size_bytes);
 
-  size_t buffer_size = bitmap_header.width * bitmap_header.height * 3;
+  size_t buffer_size = bitmap_header.width * bitmap_header.height * kRGBDims;
   float *hwc_bin_image = new (std::nothrow) float[buffer_size];
   if (hwc_bin_image == nullptr) {
     free(bmp_image);
@@ -95,14 +99,16 @@ static char *ReadBitmapFile(const std::string &filename, size_t *size) {
   for (int h = 0; h < bitmap_header.height; h++) {
     for (int w = 0; w < bitmap_header.width; w++) {
       hwc_bin_image[h * hStride + w * channels + 0] =
-        (((static_cast<float>(bmp_image[(height - h - 1) * hStride + w * channels + 2])) / 255.0) - CH_MEAN[0]) /
+        (((static_cast<float>(bmp_image[(height - h - 1) * hStride + w * channels + kBGRDim])) / kRGBMAX) -
+         CH_MEAN[0]) /
         CH_STD[0];
       hwc_bin_image[h * hStride + w * channels + 1] =
-        (((static_cast<float>(bmp_image[(height - h - 1) * hStride + w * channels + 1])) / 255.0) - CH_MEAN[1]) /
+        (((static_cast<float>(bmp_image[(height - h - 1) * hStride + w * channels + 1])) / kRGBMAX) - CH_MEAN[1]) /
         CH_STD[1];
-      hwc_bin_image[h * hStride + w * channels + 2] =
-        (((static_cast<float>(bmp_image[(height - h - 1) * hStride + w * channels + 0])) / 255.0) - CH_MEAN[2]) /
-        CH_STD[2];
+      hwc_bin_image[h * hStride + w * channels + kBGRDim] =
+        (((static_cast<float>(bmp_image[(height - h - 1) * hStride + w * channels + 0])) / kRGBMAX) -
+         CH_MEAN[kBGRDim]) /
+        CH_STD[kBGRDim];
     }
   }
 
@@ -190,7 +196,7 @@ void DataSet::InitializeBMPFoldersDatabase(std::string dpath) {
 std::vector<FileTuple> DataSet::ReadDir(const std::string dpath) {
   std::vector<FileTuple> vec;
   struct dirent *entry = nullptr;
-  num_of_classes_ = 10;
+  num_of_classes_ = kClassNum;
   for (int class_id = 0; class_id < num_of_classes_; class_id++) {
     std::string dirname = dpath + "/" + std::to_string(class_id);
     DIR *dp = opendir(dirname.c_str());
diff --git a/mindspore/lite/examples/transfer_learning/src/net_runner.cc b/mindspore/lite/examples/transfer_learning/src/net_runner.cc
index d341720..449fc54 100644
--- a/mindspore/lite/examples/transfer_learning/src/net_runner.cc
+++ b/mindspore/lite/examples/transfer_learning/src/net_runner.cc
@@ -15,9 +15,9 @@
  */
 
 #include "src/net_runner.h"
-#include <math.h>
 #include <getopt.h>
 #include <algorithm>
+#include <cmath>
 #include <cstring>
 #include <fstream>
 #include <iostream>
@@ -26,6 +26,9 @@
 #include "src/utils.h"
 
 static unsigned int seed = time(NULL);
+constexpr int kBatchNum = 20;
+constexpr int kPrintNum = 10;
+constexpr float kThreshold = 0.9f;
 
 // Definition of callback function after forwarding operator.
 bool after_callback(const std::vector<mindspore::tensor::MSTensor *> &after_inputs,
@@ -35,7 +38,7 @@ bool after_callback(const std::vector<mindspore::tensor::MSTensor *> &after_inpu
   for (size_t i = 0; i < after_inputs.size(); i++) {
     int num2p = (after_inputs.at(i)->ElementsNum());
     std::cout << "in" << i << "(" << num2p << "): ";
-    if (num2p > 10) num2p = 10;
+    if (num2p > kPrintNum) num2p = kPrintNum;
     if (after_inputs.at(i)->data_type() == mindspore::kNumberTypeInt32) {
       auto d = reinterpret_cast<int *>(after_inputs.at(i)->MutableData());
       for (int j = 0; j < num2p; j++) {
@@ -53,7 +56,7 @@ bool after_callback(const std::vector<mindspore::tensor::MSTensor *> &after_inpu
     auto d = reinterpret_cast<float *>(after_outputs.at(i)->MutableData());
     int num2p = (after_outputs.at(i)->ElementsNum());
     std::cout << "ou" << i << "(" << num2p << "): ";
-    if (num2p > 10) num2p = 10;
+    if (num2p > kPrintNum) num2p = kPrintNum;
     for (int j = 0; j < num2p; j++) {
       std::cout << d[j] << ", ";
     }
@@ -72,7 +75,7 @@ void NetRunner::InitAndFigureInputs() {
   context.thread_num_ = 1;
 
   session_ = mindspore::session::LiteSession::CreateTransferSession(ms_backbone_file_, ms_head_file_, &context);
-  MS_ASSERT(nullptr != session_);
+  MS_ASSERT(session_ != nullptr);
 
   auto inputs = session_->GetInputs();
   MS_ASSERT(inputs.size() > 1);
@@ -108,7 +111,8 @@ std::vector<int> NetRunner::FillInputData(const std::vector<DataLabelTuple> &dat
   std::fill(labels, labels + inputs.at(label_index_)->ElementsNum(), 0.f);
   for (int i = 0; i < batch_size_; i++) {
     if (serially >= 0) {
-      idx = ++idx % total_size;
+      auto reminder = ++idx % total_size;
+      idx = reminder;
     } else {
       idx = rand_r(&seed) % total_size;
     }
@@ -191,13 +195,13 @@ int NetRunner::TrainLoop() {
     }
 
     std::cout << i + 1 << ": Loss is " << loss << " [min=" << min_loss << "]" << std::endl;
-    if ((i + 1) % 20 == 0) {
+    if ((i + 1) % kBatchNum == 0) {
       session_->Eval();
       float acc = CalculateAccuracy(ds_.test_data(), session_);
       session_->Train();
       if (max_acc < acc) max_acc = acc;
       std::cout << "accuracy on test data = " << acc << " max accuracy = " << max_acc << std::endl;
-      if (acc > 0.9) return 0;
+      if (acc > kThreshold) return 0;
     }
   }
   return 0;
diff --git a/mindspore/lite/include/train/accuracy_metrics.h b/mindspore/lite/include/train/accuracy_metrics.h
index e3822fd..9dfa451 100644
--- a/mindspore/lite/include/train/accuracy_metrics.h
+++ b/mindspore/lite/include/train/accuracy_metrics.h
@@ -41,6 +41,7 @@ class AccuracyMetrics : public Metrics {
   std::vector<int> output_indexes_ = {0};
   float total_accuracy_ = 0.0;
   float total_steps_ = 0.0;
+  friend class ClassificationTrainAccuracyMonitor;
 };
 
 }  // namespace lite
diff --git a/mindspore/lite/include/train/classification_train_accuracy_monitor.h b/mindspore/lite/include/train/classification_train_accuracy_monitor.h
index 5c85592..3df8af9 100644
--- a/mindspore/lite/include/train/classification_train_accuracy_monitor.h
+++ b/mindspore/lite/include/train/classification_train_accuracy_monitor.h
@@ -16,6 +16,7 @@
 #ifndef MINDSPORE_LITE_INCLUDE_TRAIN_CLASSIFICATION_TRAIN_ACCURACY_MONITOR_H_
 #define MINDSPORE_LITE_INCLUDE_TRAIN_CLASSIFICATION_TRAIN_ACCURACY_MONITOR_H_
 #include <vector>
+#include <memory>
 #include <string>
 #include <utility>
 #include <climits>
@@ -44,9 +45,7 @@ class ClassificationTrainAccuracyMonitor : public session::TrainLoopCallBack {
 
  private:
   std::vector<GraphPoint> accuracies_;
-  int accuracy_metrics_ = METRICS_CLASSIFICATION;
-  std::vector<int> input_indexes_ = {1};
-  std::vector<int> output_indexes_ = {0};
+  std::shared_ptr<AccuracyMetrics> accuracy_metrics_;
   int print_every_n_ = 0;
 };
 
diff --git a/mindspore/lite/src/huffman_decode.cc b/mindspore/lite/src/huffman_decode.cc
index 44a0dc3..92efb73 100644
--- a/mindspore/lite/src/huffman_decode.cc
+++ b/mindspore/lite/src/huffman_decode.cc
@@ -19,7 +19,7 @@
 
 namespace mindspore {
 namespace lite {
-STATUS HuffmanDecode::DoHuffmanDecode(const std::string &input_str, void *decoded_data) {
+STATUS HuffmanDecode::DoHuffmanDecode(const std::string &input_str, void *decoded_data, size_t data_len) {
   if (decoded_data == nullptr) {
     MS_LOG(ERROR) << "decoded_data is nullptr.";
     return RET_ERROR;
@@ -57,8 +57,12 @@ STATUS HuffmanDecode::DoHuffmanDecode(const std::string &input_str, void *decode
   }
 
   size_t len = huffman_decoded_str.length();
-  memcpy(decoded_data, huffman_decoded_str.c_str(), len);
-
+  if (data_len >= len) {
+    memcpy(decoded_data, huffman_decoded_str.c_str(), len);
+  } else {
+    FreeHuffmanNodeTree(root);
+    return RET_ERROR;
+  }
   FreeHuffmanNodeTree(root);
   return RET_OK;
 }
@@ -173,6 +177,5 @@ void HuffmanDecode::FreeHuffmanNodeTree(HuffmanNodePtr root) {
     delete (cur_node);
   }
 }
-
 }  // namespace lite
 }  // namespace mindspore
diff --git a/mindspore/lite/src/huffman_decode.h b/mindspore/lite/src/huffman_decode.h
index 0495fd0..587af17 100644
--- a/mindspore/lite/src/huffman_decode.h
+++ b/mindspore/lite/src/huffman_decode.h
@@ -42,7 +42,7 @@ class HuffmanDecode {
  public:
   virtual ~HuffmanDecode() = default;
 
-  static STATUS DoHuffmanDecode(const std::string &input_str, void *decoded_data);
+  static STATUS DoHuffmanDecode(const std::string &input_str, void *decoded_data, size_t data_len);
 
  private:
   HuffmanDecode() = default;
diff --git a/mindspore/lite/src/ops/populate/arithmetic_populate.cc b/mindspore/lite/src/ops/populate/arithmetic_populate.cc
index f32f601..fe7ce3d 100644
--- a/mindspore/lite/src/ops/populate/arithmetic_populate.cc
+++ b/mindspore/lite/src/ops/populate/arithmetic_populate.cc
@@ -32,6 +32,7 @@ using mindspore::schema::PrimitiveType_Mod;
 using mindspore::schema::PrimitiveType_NotEqual;
 using mindspore::schema::PrimitiveType_RealDiv;
 using mindspore::schema::PrimitiveType_SquaredDifference;
+using mindspore::schema::PrimitiveType_BiasAddGrad;
 
 namespace mindspore {
 namespace lite {
@@ -79,5 +80,6 @@ REG_POPULATE(PrimitiveType_FloorDiv, PopulateArithmetic, SCHEMA_CUR)
 REG_POPULATE(PrimitiveType_FloorMod, PopulateArithmetic, SCHEMA_CUR)
 REG_POPULATE(PrimitiveType_Mod, PopulateArithmetic, SCHEMA_CUR)
 REG_POPULATE(PrimitiveType_SquaredDifference, PopulateArithmetic, SCHEMA_CUR)
+REG_POPULATE(PrimitiveType_BiasAddGrad, PopulateArithmetic, SCHEMA_CUR)
 }  // namespace lite
 }  // namespace mindspore
diff --git a/mindspore/lite/src/ops/populate/bias_grad_populate.cc b/mindspore/lite/src/ops/populate/bias_grad_populate.cc
deleted file mode 100644
index edcfec3..0000000
--- a/mindspore/lite/src/ops/populate/bias_grad_populate.cc
+++ /dev/null
@@ -1,39 +0,0 @@
-/**
- * Copyright 2019-2021 Huawei Technologies Co., Ltd
- *
- * Licensed under the Apache License, Version 2.0 (the "License");
- * you may not use this file except in compliance with the License.
- * You may obtain a copy of the License at
- *
- * http://www.apache.org/licenses/LICENSE-2.0
- *
- * Unless required by applicable law or agreed to in writing, software
- * distributed under the License is distributed on an "AS IS" BASIS,
- * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
- * See the License for the specific language governing permissions and
- * limitations under the License.
- */
-#include "src/ops/populate/populate_register.h"
-#include "nnacl/arithmetic.h"
-using mindspore::schema::PrimitiveType_BiasAddGrad;
-
-namespace mindspore {
-namespace lite {
-OpParameter *PopulateBiasAddGradParameter(const void *prim) {
-  auto primitive = static_cast<const schema::Primitive *>(prim);
-  MS_ASSERT(primitive != nullptr);
-
-  auto *param = reinterpret_cast<ArithmeticParameter *>(malloc(sizeof(ArithmeticParameter)));
-  if (param == nullptr) {
-    MS_LOG(ERROR) << "malloc ArithmeticParameter failed.";
-    return nullptr;
-  }
-  memset(param, 0, sizeof(ArithmeticParameter));
-
-  param->op_parameter_.type_ = primitive->value_type();
-  return reinterpret_cast<OpParameter *>(param);
-}
-
-REG_POPULATE(PrimitiveType_BiasAddGrad, PopulateBiasAddGradParameter, SCHEMA_CUR);
-}  // namespace lite
-}  // namespace mindspore
diff --git a/mindspore/lite/src/train/accuracy_metrics.cc b/mindspore/lite/src/train/accuracy_metrics.cc
index 6b79088..6d6a5c5 100644
--- a/mindspore/lite/src/train/accuracy_metrics.cc
+++ b/mindspore/lite/src/train/accuracy_metrics.cc
@@ -22,7 +22,6 @@
 
 namespace mindspore {
 namespace lite {
-
 AccuracyMetrics::AccuracyMetrics(int accuracy_metrics, const std::vector<int> &input_indexes,
                                  const std::vector<int> &output_indexes)
     : Metrics() {
@@ -66,6 +65,5 @@ float AccuracyMetrics::Eval() {
 
   return (total_accuracy_ / total_steps_);
 }
-
 }  // namespace lite
 }  // namespace mindspore
diff --git a/mindspore/lite/src/train/accuracy_monitor.cc b/mindspore/lite/src/train/accuracy_monitor.cc
index a9aabdc..0cb3b5d 100644
--- a/mindspore/lite/src/train/accuracy_monitor.cc
+++ b/mindspore/lite/src/train/accuracy_monitor.cc
@@ -29,7 +29,6 @@
 
 namespace mindspore {
 namespace lite {
-
 void AccuracyMonitor::Begin(const session::TrainLoopCallBackData &cb_data) {
   if (cb_data.epoch_ == 0) accuracies_.clear();
 }
@@ -40,6 +39,5 @@ int AccuracyMonitor::EpochEnd(const session::TrainLoopCallBackData &cb_data) {
   accuracies_.push_back(std::make_pair(cb_data.epoch_, 0.0));
   return mindspore::session::RET_CONTINUE;
 }
-
 }  // namespace lite
 }  // namespace mindspore
diff --git a/mindspore/lite/src/train/classification_train_accuracy_monitor.cc b/mindspore/lite/src/train/classification_train_accuracy_monitor.cc
index d398429..d806902 100644
--- a/mindspore/lite/src/train/classification_train_accuracy_monitor.cc
+++ b/mindspore/lite/src/train/classification_train_accuracy_monitor.cc
@@ -26,21 +26,10 @@ using mindspore::WARNING;
 
 namespace mindspore {
 namespace lite {
-
 ClassificationTrainAccuracyMonitor::ClassificationTrainAccuracyMonitor(int print_every_n, int accuracy_metrics,
                                                                        const std::vector<int> &input_indexes,
                                                                        const std::vector<int> &output_indexes) {
-  if (input_indexes.size() == output_indexes.size()) {
-    input_indexes_ = input_indexes;
-    output_indexes_ = output_indexes;
-  } else {
-    MS_LOG(WARNING) << "input to output mapping vectors sizes do not match";
-  }
-  if (accuracy_metrics != METRICS_CLASSIFICATION) {
-    MS_LOG(WARNING) << "Only classification metrics is supported";
-  } else {
-    accuracy_metrics_ = accuracy_metrics;
-  }
+  accuracy_metrics_ = std::make_shared<AccuracyMetrics>(accuracy_metrics, input_indexes, output_indexes);
   print_every_n_ = print_every_n;
 }
 
@@ -59,8 +48,8 @@ void ClassificationTrainAccuracyMonitor::EpochBegin(const session::TrainLoopCall
 int ClassificationTrainAccuracyMonitor::EpochEnd(const session::TrainLoopCallBackData &cb_data) {
   if (cb_data.step_ > 0) accuracies_.at(cb_data.epoch_).second /= static_cast<float>(cb_data.step_ + 1);
   if ((cb_data.epoch_ + 1) % print_every_n_ == 0) {
-    std::cout << "Epoch (" << cb_data.epoch_ + 1 << "):\tTraining Accuracy is " << accuracies_.at(cb_data.epoch_).second
-              << std::endl;
+    std::cout << "Epoch (" << (cb_data.epoch_ + 1) << "):\tTraining Accuracy is "
+              << accuracies_.at(cb_data.epoch_).second << std::endl;
   }
   return mindspore::session::RET_CONTINUE;
 }
@@ -70,21 +59,22 @@ void ClassificationTrainAccuracyMonitor::StepEnd(const session::TrainLoopCallBac
   auto outputs = cb_data.session_->GetPredictions();
 
   float accuracy = 0.0;
-  for (unsigned int i = 0; i < input_indexes_.size(); i++) {
-    if ((inputs.size() <= static_cast<unsigned int>(input_indexes_[i])) ||
-        (outputs.size() <= static_cast<unsigned int>(output_indexes_[i]))) {
-      MS_LOG(WARNING) << "indices " << input_indexes_[i] << "/" << output_indexes_[i]
+  auto input_indexes = accuracy_metrics_->input_indexes_;
+  auto output_indexes = accuracy_metrics_->output_indexes_;
+  for (unsigned int i = 0; i < input_indexes.size(); i++) {
+    if ((inputs.size() <= static_cast<unsigned int>(input_indexes[i])) ||
+        (outputs.size() <= static_cast<unsigned int>(output_indexes[i]))) {
+      MS_LOG(WARNING) << "indices " << input_indexes[i] << "/" << output_indexes[i]
                       << " is outside of input/output range";
       return;
     }
-    if (inputs.at(input_indexes_[i])->data_type() == kNumberTypeInt32) {
-      accuracy += CalculateSparseClassification(inputs.at(input_indexes_[i]), outputs.at(output_indexes_[i]));
+    if (inputs.at(input_indexes[i])->data_type() == kNumberTypeInt32) {
+      accuracy += CalculateSparseClassification(inputs.at(input_indexes[i]), outputs.at(output_indexes[i]));
     } else {
-      accuracy += CalculateOneHotClassification(inputs.at(input_indexes_[i]), outputs.at(output_indexes_[i]));
+      accuracy += CalculateOneHotClassification(inputs.at(input_indexes[i]), outputs.at(output_indexes[i]));
     }
   }
   accuracies_.at(cb_data.epoch_).second += accuracy;
 }
-
 }  // namespace lite
 }  // namespace mindspore
diff --git a/mindspore/lite/src/train/loss_monitor.cc b/mindspore/lite/src/train/loss_monitor.cc
index bd3e529..60637f9 100644
--- a/mindspore/lite/src/train/loss_monitor.cc
+++ b/mindspore/lite/src/train/loss_monitor.cc
@@ -26,7 +26,6 @@
 
 namespace mindspore {
 namespace lite {
-
 void LossMonitor::Begin(const session::TrainLoopCallBackData &cb_data) {
   if (cb_data.epoch_ == 0) losses_.clear();
 }
@@ -42,7 +41,7 @@ void LossMonitor::EpochBegin(const session::TrainLoopCallBackData &cb_data) {
 int LossMonitor::EpochEnd(const session::TrainLoopCallBackData &cb_data) {
   if (cb_data.step_ > 0) losses_.at(cb_data.epoch_).second /= static_cast<float>(cb_data.step_ + 1);
   if (print_every_n_ > 0) {
-    std::cout << "Epoch (" << cb_data.epoch_ + 1 << "):\tLoss is " << losses_.at(cb_data.epoch_).second << std::endl;
+    std::cout << "Epoch (" << (cb_data.epoch_ + 1) << "):\tLoss is " << losses_.at(cb_data.epoch_).second << std::endl;
   }
   return mindspore::session::RET_CONTINUE;
 }
@@ -54,12 +53,11 @@ void LossMonitor::StepEnd(const session::TrainLoopCallBackData &cb_data) {
       auto loss = reinterpret_cast<float *>(it->second->MutableData());
       losses_.at(cb_data.epoch_).second += loss[0];
       if ((cb_data.step_ + 1) % print_every_n_ == 0)
-        std::cout << cb_data.epoch_ + 1 << "." << cb_data.step_ + 1 << ":\tLoss is " << loss[0] << std::endl;
+        std::cout << (cb_data.epoch_ + 1) << "." << (cb_data.step_ + 1) << ":\tLoss is " << loss[0] << std::endl;
       return;
     }
   }
   MS_LOG(WARNING) << "Model does not have a loss output tensor of size 1";
 }
-
 }  // namespace lite
 }  // namespace mindspore
diff --git a/mindspore/lite/src/train/lr_scheduler.cc b/mindspore/lite/src/train/lr_scheduler.cc
index ea9d074..d9713c3 100644
--- a/mindspore/lite/src/train/lr_scheduler.cc
+++ b/mindspore/lite/src/train/lr_scheduler.cc
@@ -29,7 +29,6 @@
 
 namespace mindspore {
 namespace lite {
-
 int MultiplicativeLRLambda(float *lr, int epoch, void *lr_cb_data) {
   if ((lr == nullptr) || (lr_cb_data == nullptr)) {
     MS_LOG(ERROR) << "nullptr passed as input to MultiplicativeLRLambda";
@@ -70,6 +69,5 @@ int LRScheduler::EpochEnd(const session::TrainLoopCallBackData &cb_data) {
   }
   return mindspore::session::RET_CONTINUE;
 }
-
 }  // namespace lite
 }  // namespace mindspore
diff --git a/mindspore/lite/src/train/train_loop.cc b/mindspore/lite/src/train/train_loop.cc
index 93ab442..4f3f904 100644
--- a/mindspore/lite/src/train/train_loop.cc
+++ b/mindspore/lite/src/train/train_loop.cc
@@ -25,7 +25,6 @@
 
 namespace mindspore {
 namespace lite {
-
 using dataset::Dataset;
 using dataset::Iterator;
 using dataset::MSTensorVec;
@@ -132,28 +131,8 @@ int TrainLoop::LoadData(std::vector<tensor::MSTensor *> inputs, dataset::MSTenso
   }
 
   for (unsigned int i = 0; i < num_of_inputs; i++) {
-    unsigned char *input_data = reinterpret_cast<unsigned char *>(inputs.at(i)->MutableData());
-    const unsigned char *row_data = reinterpret_cast<const unsigned char *>(row_vec->at(i).MutableData());
-    auto data_size = row_vec->at(i).DataSize();
-    if (data_size != inputs.at(i)->Size()) {
-      MS_LOG(WARNING) << "Model Input tensor " << i << " size (" << inputs.at(i)->Size()
-                      << ") does not match dataset size (" << data_size << ")\n";
-      return RET_STOP_TRAINING;
-    }
-    std::copy(row_data, row_data + data_size, input_data);
-  }
-  return RET_OK;
-}
-
-int TrainLoop::LoadPartialData(std::vector<tensor::MSTensor *> inputs, dataset::MSTensorVec *row_vec) {
-  auto num_of_inputs = inputs.size();
-  if ((num_of_inputs == 0) || (row_vec == nullptr) || (num_of_inputs < row_vec->size())) {
-    return RET_STOP_TRAINING;
-  }
-
-  for (unsigned int i = 0; i < row_vec->size(); i++) {
-    unsigned char *input_data = reinterpret_cast<unsigned char *>(inputs.at(i)->MutableData());
-    const unsigned char *row_data = reinterpret_cast<const unsigned char *>(row_vec->at(i).MutableData());
+    auto *input_data = reinterpret_cast<unsigned char *>(inputs.at(i)->MutableData());
+    const auto *row_data = reinterpret_cast<const unsigned char *>(row_vec->at(i).MutableData());
     auto data_size = row_vec->at(i).DataSize();
     if (data_size != inputs.at(i)->Size()) {
       MS_LOG(WARNING) << "Model Input tensor " << i << " size (" << inputs.at(i)->Size()
@@ -164,12 +143,10 @@ int TrainLoop::LoadPartialData(std::vector<tensor::MSTensor *> inputs, dataset::
   }
   return RET_OK;
 }
-
 }  // namespace lite
 
 session::TrainLoop *session::TrainLoop::CreateTrainLoop(session::LiteSession *train_session) {
   auto loop = new (std::nothrow) lite::TrainLoop(train_session);
   return loop;
 }
-
 }  // namespace mindspore
diff --git a/mindspore/lite/src/train/train_loop.h b/mindspore/lite/src/train/train_loop.h
index e35a71a..40a63da 100644
--- a/mindspore/lite/src/train/train_loop.h
+++ b/mindspore/lite/src/train/train_loop.h
@@ -63,7 +63,6 @@ class TrainLoop : virtual public session::TrainLoop {
 
  protected:
   static int LoadData(std::vector<tensor::MSTensor *> inputs, dataset::MSTensorVec *dataset_vec);
-  static int LoadPartialData(std::vector<tensor::MSTensor *> inputs, dataset::MSTensorVec *dataset_vec);
 
   session::LiteSession *train_session_ = nullptr;
   unsigned int epoch_ = 0;
diff --git a/mindspore/lite/src/train/train_populate_parameter_v0.cc b/mindspore/lite/src/train/train_populate_parameter_v0.cc
index 072dfb0..80c907e 100644
--- a/mindspore/lite/src/train/train_populate_parameter_v0.cc
+++ b/mindspore/lite/src/train/train_populate_parameter_v0.cc
@@ -592,7 +592,6 @@ OpParameter *PopulateArithmeticGradParameter(const void *primitive) {
   }
   return reinterpret_cast<OpParameter *>(arithmetic_param);
 }
-
 }  // namespace
 
 void PopulateTrainV0Parameters() {
@@ -665,5 +664,4 @@ void PopulateTrainV0Parameters() {
   lite::Registry g_sigmoidCrossEntropyWithLogitsGradRegistry(
     schema::v0::PrimitiveType_SigmoidCrossEntropyWithLogitsGrad, DefaultPopulateParameter, mindspore::lite::SCHEMA_V0);
 }
-
 }  // namespace mindspore::kernel
diff --git a/mindspore/lite/src/weight_decoder.cc b/mindspore/lite/src/weight_decoder.cc
index 86492d7..ece9582 100644
--- a/mindspore/lite/src/weight_decoder.cc
+++ b/mindspore/lite/src/weight_decoder.cc
@@ -255,7 +255,7 @@ int WeightDecoder::DecodeHuffmanCode(const schema::Tensor &src_tensor, lite::Ten
   }
   auto dst_data = dst_tensor->data_c();
   MS_ASSERT(dst_data != nullptr);
-  ret = HuffmanDecode::DoHuffmanDecode(encode_str, dst_data);
+  ret = HuffmanDecode::DoHuffmanDecode(encode_str, dst_data, dst_tensor->Size());
   if (ret != RET_OK) {
     MS_LOG(ERROR) << "DoHuffmanDecode failed.";
     return ret;
-- 
2.7.4

